In [ ]:
!pip install librosa opencv-python numpy matplotlib tensorflow

In [ ]:
from google.colab import files
import zipfile
import os

# Upload your zip file (this will open a file dialog)
uploaded = files.upload()

# Extract the uploaded zip file
for filename in uploaded.keys():
    zip_path = f"/content/{filename}"
    extract_path = "/content/audio_data"
    os.makedirs(extract_path, exist_ok=True)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

# Show top-level folders inside the extracted path
print("Extracted folders in /content/audio_data:")
print(os.listdir(extract_path))

In [ ]:
from google.colab import files
import zipfile

uploaded = files.upload()  # Upload your dataset.zip again

for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('/content/dataset')
        print(f"{filename} extracted to /content/dataset")


In [ ]:
import os

for root, dirs, files in os.walk("/content/audio_data"):
    level = root.replace("/content/audio_data", "").count(os.sep)
    indent = " " * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    for f in dirs:
        print(f"{indent}    {f}/")

In [ ]:
import os

# List contents of /content to see where files might have been placed
print("Contents of /content:")
print(os.listdir("/content"))

In [ ]:
import zipfile
import os

# Path to the zip file
zip_path = "/content/Three-classes-classify-20250512T134602Z-1-001.zip"
extract_path = "/content/dataset"

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# List contents of extracted folder to check if the classes are there
print("Contents of extracted dataset:")
print(os.listdir(extract_path))

In [ ]:
# List the contents inside the Three-classes-classify folder
extracted_folder = "/content/dataset/Three-classes-classify"
print("Contents of /content/dataset/Three-classes-classify:")
print(os.listdir(extracted_folder))

In [ ]:
# List the contents inside the nested Three-classes-classify folder
nested_folder = "/content/dataset/Three-classes-classify/Three-classes-classify"
print("Contents of /content/dataset/Three-classes-classify/Three-classes-classify:")
print(os.listdir(nested_folder))

In [ ]:
import shutil

# Define source and destination directories
source_dir = "/content/dataset/Three-classes-classify/Three-classes-classify"
dest_dir = "/content/audio_data"

# Move class folders to audio_data
for folder_name in os.listdir(source_dir):
    folder_path = os.path.join(source_dir, folder_name)
    if os.path.isdir(folder_path):
        shutil.move(folder_path, dest_dir)

# List contents of /content/audio_data to confirm move
print("Contents of /content/audio_data after move:")
print(os.listdir(dest_dir))

In [ ]:
# Check for audio files in each class folder
for folder_name in os.listdir("/content/audio_data"):
    folder_path = os.path.join("/content/audio_data", folder_name)
    if os.path.isdir(folder_path):
        print(f"Files in {folder_name}:")
        print(os.listdir(folder_path)[:5])  # Print first 5 files for each class

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os

# Function to convert audio file to spectrogram
def audio_to_spectrogram(audio_path, image_path):
    # Load audio file
    y, sr = librosa.load(audio_path, sr=None)

    # Generate spectrogram
    D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)

    # Plot the spectrogram
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='log')
    plt.colorbar(format='%+2.0f dB')
    plt.tight_layout()

    # Save spectrogram as image
    plt.savefig(image_path, bbox_inches='tight', pad_inches=0)
    plt.close()

# Directory to save spectrograms
spectrogram_dir = '/content/spectrograms'
os.makedirs(spectrogram_dir, exist_ok=True)

# Loop through class folders and process audio files
for folder_name in os.listdir("/content/audio_data"):
    folder_path = os.path.join("/content/audio_data", folder_name)
    if os.path.isdir(folder_path):
        # Create a sub-folder for each class in spectrogram_dir
        class_spectrogram_dir = os.path.join(spectrogram_dir, folder_name)
        os.makedirs(class_spectrogram_dir, exist_ok=True)

        # Process each audio file in the class folder
        for audio_file in os.listdir(folder_path):
            if audio_file.endswith(".wav"):
                audio_path = os.path.join(folder_path, audio_file)
                spectrogram_path = os.path.join(class_spectrogram_dir, audio_file.replace(".wav", ".png"))
                audio_to_spectrogram(audio_path, spectrogram_path)

print("Spectrogram generation complete!")

In [ ]:
import shutil
from sklearn.model_selection import train_test_split
import os

# Directory where the spectrograms are stored
spectrogram_dir = '/content/spectrograms'

# Create directories for training and validation sets
train_dir = '/content/spectrograms/train'
val_dir = '/content/spectrograms/val'

# Create the train and validation directories if they do not exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Create subdirectories for each class within train and validation directories
for class_name in ['Human-Voice', 'Human-Voicemail', 'Machine-Voicemail']:
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)

# Split the data into training and validation sets
for class_name in ['Human-Voice', 'Human-Voicemail', 'Machine-Voicemail']:
    class_path = os.path.join(spectrogram_dir, class_name)
    files = os.listdir(class_path)

    # Split the files into train and validation sets
    train_files, val_files = train_test_split(files, test_size=0.2, random_state=42)

    # Move files to respective directories
    for file in train_files:
        shutil.move(os.path.join(class_path, file), os.path.join(train_dir, class_name, file))

    for file in val_files:
        shutil.move(os.path.join(class_path, file), os.path.join(val_dir, class_name, file))

# Check the new directory structure
print(f"Train directory: {os.listdir(train_dir)}")
print(f"Validation directory: {os.listdir(val_dir)}")

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Set image size (VGG16 expects 224x224 input size)
IMG_SIZE = 224

# Create an ImageDataGenerator for loading and augmenting images
datagen = ImageDataGenerator(rescale=1./255)

# Path to the spectrogram directory
spectrogram_dir = '/content/spectrograms'

# Prepare the image dataset using flow_from_directory
train_data_gen = datagen.flow_from_directory(
    spectrogram_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

# Split the data into training and validation sets
# train_data_gen already takes care of shuffling, we just need to specify validation split
val_data_gen = datagen.flow_from_directory(
    spectrogram_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
    subset='validation'
)

print("Data preparation complete!")

In [ ]:
import os

# Define train and validation directories
train_dir = '/content/spectrograms/train'
val_dir = '/content/spectrograms/val'

# Check the contents of each class subdirectory in train and validation directories
for dataset_dir, dataset_name in zip([train_dir, val_dir], ["Train", "Validation"]):
    print(f"\n{dataset_name} Set:")
    for class_name in ['Human-Voice', 'Human-Voicemail', 'Machine-Voicemail']:
        class_dir = os.path.join(dataset_dir, class_name)
        if os.path.exists(class_dir):
            print(f"{class_name}: {len(os.listdir(class_dir))} files")
        else:
            print(f"{class_name}: Directory does not exist")

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation and rescaling
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Rescale pixel values to [0, 1]
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Load images from directories
train_generator = train_datagen.flow_from_directory(
    '/content/spectrograms/train',
    target_size=(224, 224),  # Resize images to VGG16 input size
    batch_size=32,
    class_mode='categorical'  # We have 3 classes
)

val_generator = val_datagen.flow_from_directory(
    '/content/spectrograms/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models

# Load VGG16 model with pre-trained weights (exclude top layer)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers to prevent training during initial fine-tuning
base_model.trainable = False

# Add custom top layers for our specific problem
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')  # 3 output classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,  # Adjust epochs as needed
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size
)

In [ ]:
# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(val_generator, steps=val_generator.samples // val_generator.batch_size)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

In [ ]:
# Predict on new spectrogram images
import numpy as np
from tensorflow.keras.preprocessing import image

# Load a new image for prediction
img_path = '/content/20250422-211509-17348559018.png'  # Replace with the path of the new image
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0  # Rescale the image
img_array = np.expand_dims(img_array, axis=0)

# Make prediction
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions, axis=1)

# Map prediction to class names
class_names = ['Human-Voice', 'Human-Voicemail', 'Machine-Voicemail']
print(f"Predicted Class: {class_names[predicted_class[0]]}")